# S16 — Flow Control & State Management

## 🎯 Objectifs
- Implémenter la persistence d'état avec SQLite
- Gérer les checkpoints et rollbacks
- Orchestrer des workflows avec gestion d'état
- Implémenter la reprise après erreur

## 📋 Contenu
1. Introduction à la gestion d'état
2. Persistence avec SQLite
3. Système de checkpoints
4. Rollback et récupération d'erreurs
5. Workflows stateful complexes

## 1. Installation et Configuration

In [ ]:
# Installation des dépendances
# SQLite est inclus avec Python, pas besoin d'installation supplémentaire

In [ ]:
import sqlite3
import json
import pickle
from datetime import datetime
from typing import Dict, Any, List, Optional
from dataclasses import dataclass, asdict
import os
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Bibliothèques importées")

## 2. State Manager avec SQLite

### 2.1 Définition de l'état

In [ ]:
@dataclass
class WorkflowState:
    """État d'un workflow"""
    workflow_id: str
    status: str  # 'running', 'completed', 'failed', 'paused'
    current_step: str
    data: Dict[str, Any]
    created_at: str
    updated_at: str
    metadata: Dict[str, Any]
    
    def to_dict(self) -> Dict:
        return asdict(self)

print("✅ WorkflowState défini")

### 2.2 StateManager avec SQLite

In [ ]:
class StateManager:
    """Gestionnaire d'état avec persistence SQLite"""
    
    def __init__(self, db_path: str = "workflow_state.db"):
        self.db_path = db_path
        self._init_db()
    
    def _init_db(self):
        """Initialiser la base de données"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Table pour les états
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS workflow_states (
                workflow_id TEXT PRIMARY KEY,
                status TEXT NOT NULL,
                current_step TEXT NOT NULL,
                data TEXT NOT NULL,
                created_at TEXT NOT NULL,
                updated_at TEXT NOT NULL,
                metadata TEXT
            )
        """)
        
        # Table pour les checkpoints
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS checkpoints (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                workflow_id TEXT NOT NULL,
                checkpoint_name TEXT NOT NULL,
                state_data TEXT NOT NULL,
                created_at TEXT NOT NULL,
                FOREIGN KEY (workflow_id) REFERENCES workflow_states(workflow_id)
            )
        """)
        
        # Table pour l'historique
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS state_history (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                workflow_id TEXT NOT NULL,
                action TEXT NOT NULL,
                step TEXT NOT NULL,
                details TEXT,
                timestamp TEXT NOT NULL
            )
        """)
        
        conn.commit()
        conn.close()
        logger.info(f"✅ Base de données initialisée: {self.db_path}")
    
    def save_state(self, state: WorkflowState):
        """Sauvegarder l'état"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            INSERT OR REPLACE INTO workflow_states
            (workflow_id, status, current_step, data, created_at, updated_at, metadata)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (
            state.workflow_id,
            state.status,
            state.current_step,
            json.dumps(state.data),
            state.created_at,
            state.updated_at,
            json.dumps(state.metadata)
        ))
        
        # Log dans l'historique
        cursor.execute("""
            INSERT INTO state_history (workflow_id, action, step, details, timestamp)
            VALUES (?, ?, ?, ?, ?)
        """, (
            state.workflow_id,
            'save_state',
            state.current_step,
            f"Status: {state.status}",
            datetime.now().isoformat()
        ))
        
        conn.commit()
        conn.close()
        logger.info(f"💾 État sauvegardé: {state.workflow_id} - {state.current_step}")
    
    def load_state(self, workflow_id: str) -> Optional[WorkflowState]:
        """Charger l'état"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT workflow_id, status, current_step, data, created_at, updated_at, metadata
            FROM workflow_states
            WHERE workflow_id = ?
        """, (workflow_id,))
        
        row = cursor.fetchone()
        conn.close()
        
        if row:
            state = WorkflowState(
                workflow_id=row[0],
                status=row[1],
                current_step=row[2],
                data=json.loads(row[3]),
                created_at=row[4],
                updated_at=row[5],
                metadata=json.loads(row[6]) if row[6] else {}
            )
            logger.info(f"📂 État chargé: {workflow_id}")
            return state
        
        logger.warning(f"⚠️  État non trouvé: {workflow_id}")
        return None
    
    def create_checkpoint(self, workflow_id: str, checkpoint_name: str):
        """Créer un checkpoint"""
        state = self.load_state(workflow_id)
        if not state:
            raise ValueError(f"Workflow {workflow_id} non trouvé")
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            INSERT INTO checkpoints (workflow_id, checkpoint_name, state_data, created_at)
            VALUES (?, ?, ?, ?)
        """, (
            workflow_id,
            checkpoint_name,
            json.dumps(state.to_dict()),
            datetime.now().isoformat()
        ))
        
        # Log
        cursor.execute("""
            INSERT INTO state_history (workflow_id, action, step, details, timestamp)
            VALUES (?, ?, ?, ?, ?)
        """, (
            workflow_id,
            'create_checkpoint',
            state.current_step,
            f"Checkpoint: {checkpoint_name}",
            datetime.now().isoformat()
        ))
        
        conn.commit()
        conn.close()
        logger.info(f"📌 Checkpoint créé: {checkpoint_name}")
    
    def rollback_to_checkpoint(self, workflow_id: str, checkpoint_name: str):
        """Revenir à un checkpoint"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT state_data FROM checkpoints
            WHERE workflow_id = ? AND checkpoint_name = ?
            ORDER BY created_at DESC
            LIMIT 1
        """, (workflow_id, checkpoint_name))
        
        row = cursor.fetchone()
        
        if not row:
            conn.close()
            raise ValueError(f"Checkpoint '{checkpoint_name}' non trouvé")
        
        # Restaurer l'état
        state_dict = json.loads(row[0])
        state = WorkflowState(**state_dict)
        state.updated_at = datetime.now().isoformat()
        
        conn.close()
        
        # Sauvegarder l'état restauré
        self.save_state(state)
        
        # Log
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO state_history (workflow_id, action, step, details, timestamp)
            VALUES (?, ?, ?, ?, ?)
        """, (
            workflow_id,
            'rollback',
            state.current_step,
            f"Rollback to: {checkpoint_name}",
            datetime.now().isoformat()
        ))
        conn.commit()
        conn.close()
        
        logger.info(f"⏮️  Rollback vers: {checkpoint_name}")
        return state
    
    def get_history(self, workflow_id: str) -> List[Dict]:
        """Obtenir l'historique d'un workflow"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT action, step, details, timestamp
            FROM state_history
            WHERE workflow_id = ?
            ORDER BY timestamp
        """, (workflow_id,))
        
        rows = cursor.fetchall()
        conn.close()
        
        history = [
            {
                "action": row[0],
                "step": row[1],
                "details": row[2],
                "timestamp": row[3]
            }
            for row in rows
        ]
        
        return history
    
    def list_checkpoints(self, workflow_id: str) -> List[Dict]:
        """Lister les checkpoints"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT checkpoint_name, created_at
            FROM checkpoints
            WHERE workflow_id = ?
            ORDER BY created_at
        """, (workflow_id,))
        
        rows = cursor.fetchall()
        conn.close()
        
        checkpoints = [
            {
                "name": row[0],
                "created_at": row[1]
            }
            for row in rows
        ]
        
        return checkpoints

print("✅ StateManager défini")

## 3. Workflow avec Gestion d'État

### 3.1 Définition d'un workflow simple

In [ ]:
class StatefulWorkflow:
    """Workflow avec gestion d'état et checkpoints"""
    
    def __init__(self, workflow_id: str, state_manager: StateManager):
        self.workflow_id = workflow_id
        self.state_manager = state_manager
        self.steps = ['init', 'process', 'validate', 'complete']
    
    def start(self, initial_data: Dict[str, Any]):
        """Démarrer le workflow"""
        state = WorkflowState(
            workflow_id=self.workflow_id,
            status='running',
            current_step='init',
            data=initial_data,
            created_at=datetime.now().isoformat(),
            updated_at=datetime.now().isoformat(),
            metadata={'step_index': 0}
        )
        self.state_manager.save_state(state)
        logger.info(f"🚀 Workflow démarré: {self.workflow_id}")
        return state
    
    def execute_step(self, step_name: str, force_error: bool = False):
        """Exécuter une étape"""
        state = self.state_manager.load_state(self.workflow_id)
        if not state:
            raise ValueError(f"Workflow {self.workflow_id} non trouvé")
        
        logger.info(f"▶️  Exécution de l'étape: {step_name}")
        
        # Simuler le traitement
        if force_error:
            state.status = 'failed'
            state.metadata['error'] = f"Erreur simulée à l'étape {step_name}"
            self.state_manager.save_state(state)
            raise Exception(f"Erreur à l'étape {step_name}")
        
        # Mise à jour de l'état
        state.current_step = step_name
        state.updated_at = datetime.now().isoformat()
        state.data[f'{step_name}_completed'] = True
        state.data[f'{step_name}_timestamp'] = datetime.now().isoformat()
        
        # Marquer comme complété si dernière étape
        if step_name == self.steps[-1]:
            state.status = 'completed'
        
        self.state_manager.save_state(state)
        return state
    
    def run_all_steps(self, with_checkpoints: bool = True):
        """Exécuter toutes les étapes"""
        for step in self.steps:
            self.execute_step(step)
            
            # Créer un checkpoint après chaque étape
            if with_checkpoints:
                self.state_manager.create_checkpoint(
                    self.workflow_id,
                    f"after_{step}"
                )
        
        logger.info(f"✅ Workflow terminé: {self.workflow_id}")
    
    def resume(self):
        """Reprendre le workflow depuis le dernier état"""
        state = self.state_manager.load_state(self.workflow_id)
        if not state:
            raise ValueError(f"Workflow {self.workflow_id} non trouvé")
        
        current_index = self.steps.index(state.current_step)
        remaining_steps = self.steps[current_index + 1:]
        
        logger.info(f"🔄 Reprise du workflow depuis: {state.current_step}")
        
        for step in remaining_steps:
            self.execute_step(step)
            self.state_manager.create_checkpoint(
                self.workflow_id,
                f"after_{step}"
            )

print("✅ StatefulWorkflow défini")

## 4. Démonstration: Workflow avec Checkpoints

### 4.1 Créer un workflow et exécuter avec checkpoints

In [ ]:
# Nettoyer la base de données si elle existe
if os.path.exists('workflow_state.db'):
    os.remove('workflow_state.db')

# Créer le state manager
state_manager = StateManager('workflow_state.db')

# Créer et démarrer un workflow
workflow = StatefulWorkflow('workflow_001', state_manager)
initial_state = workflow.start({
    'input': 'test data',
    'user_id': 'user_123'
})

print("\n📊 État initial:")
print(json.dumps(initial_state.to_dict(), indent=2))

In [ ]:
# Exécuter les deux premières étapes
print("\n" + "="*60)
print("Exécution des étapes 1-2")
print("="*60)

workflow.execute_step('init')
state_manager.create_checkpoint('workflow_001', 'after_init')

workflow.execute_step('process')
state_manager.create_checkpoint('workflow_001', 'after_process')

# Vérifier l'état
current_state = state_manager.load_state('workflow_001')
print(f"\n📊 État actuel: {current_state.current_step}")
print(f"Status: {current_state.status}")

### 4.2 Simuler une erreur et rollback

In [ ]:
print("\n" + "="*60)
print("Simulation d'une erreur")
print("="*60)

try:
    # Tenter d'exécuter avec erreur forcée
    workflow.execute_step('validate', force_error=True)
except Exception as e:
    print(f"\n❌ Erreur capturée: {e}")
    
    # Lister les checkpoints disponibles
    checkpoints = state_manager.list_checkpoints('workflow_001')
    print(f"\n📌 Checkpoints disponibles:")
    for cp in checkpoints:
        print(f"  - {cp['name']} (créé à {cp['created_at']})")
    
    # Rollback au dernier checkpoint
    print(f"\n⏮️  Rollback vers 'after_process'...")
    restored_state = state_manager.rollback_to_checkpoint('workflow_001', 'after_process')
    
    print(f"\n✅ État restauré:")
    print(f"  Step: {restored_state.current_step}")
    print(f"  Status: {restored_state.status}")

### 4.3 Reprendre le workflow

In [ ]:
print("\n" + "="*60)
print("Reprise du workflow")
print("="*60)

# Reprendre depuis l'état actuel
workflow.resume()

# Vérifier l'état final
final_state = state_manager.load_state('workflow_001')
print(f"\n✅ Workflow terminé!")
print(f"  Status: {final_state.status}")
print(f"  Étape finale: {final_state.current_step}")

## 5. Visualisation de l'Historique

In [ ]:
import pandas as pd

# Obtenir l'historique
history = state_manager.get_history('workflow_001')

# Créer un DataFrame
df_history = pd.DataFrame(history)

print("\n" + "="*60)
print("HISTORIQUE DU WORKFLOW")
print("="*60)
print(df_history.to_string(index=False))

In [ ]:
# Visualisation graphique
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 6))

# Extraire les données
actions = df_history['action'].tolist()
timestamps = df_history['timestamp'].tolist()
steps = df_history['step'].tolist()

# Créer un graphique de timeline
y_positions = list(range(len(actions)))

colors = {
    'save_state': '#3498db',
    'create_checkpoint': '#2ecc71',
    'rollback': '#e74c3c'
}

for i, (action, step) in enumerate(zip(actions, steps)):
    color = colors.get(action, '#95a5a6')
    ax.scatter(i, 0, s=200, c=color, alpha=0.6, edgecolors='black', linewidth=2)
    ax.text(i, -0.15, step, ha='center', va='top', fontsize=9, rotation=45)
    ax.text(i, 0.15, action, ha='center', va='bottom', fontsize=8, style='italic')

# Tracer les lignes de connexion
for i in range(len(actions) - 1):
    ax.plot([i, i+1], [0, 0], 'k-', alpha=0.3, linewidth=1)

ax.set_ylim(-0.5, 0.5)
ax.set_xlim(-0.5, len(actions) - 0.5)
ax.set_yticks([])
ax.set_xlabel('Chronologie', fontsize=12)
ax.set_title('Timeline du Workflow avec Checkpoints', fontsize=14, fontweight='bold')
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Légende
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#3498db', label='Save State'),
    Patch(facecolor='#2ecc71', label='Checkpoint'),
    Patch(facecolor='#e74c3c', label='Rollback')
]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

print("\n📊 Visualisation générée")

## 6. Cas d'Usage Avancé: Workflow avec Branches

### 6.1 Workflow avec décisions conditionnelles

In [ ]:
class ConditionalWorkflow(StatefulWorkflow):
    """Workflow avec branches conditionnelles"""
    
    def __init__(self, workflow_id: str, state_manager: StateManager):
        super().__init__(workflow_id, state_manager)
        self.steps = ['init', 'analyze', 'decision', 'path_a', 'path_b', 'complete']
    
    def execute_step(self, step_name: str, force_error: bool = False):
        state = self.state_manager.load_state(self.workflow_id)
        if not state:
            raise ValueError(f"Workflow {self.workflow_id} non trouvé")
        
        logger.info(f"▶️  Exécution: {step_name}")
        
        # Logique de décision
        if step_name == 'decision':
            # Décider du chemin basé sur les données
            value = state.data.get('analysis_result', 0)
            if value > 50:
                state.metadata['chosen_path'] = 'path_a'
                logger.info("   → Chemin A choisi (valeur > 50)")
            else:
                state.metadata['chosen_path'] = 'path_b'
                logger.info("   → Chemin B choisi (valeur <= 50)")
        
        # Mise à jour standard
        state.current_step = step_name
        state.updated_at = datetime.now().isoformat()
        state.data[f'{step_name}_completed'] = True
        
        # Ajouter des données simulées
        if step_name == 'analyze':
            import random
            state.data['analysis_result'] = random.randint(0, 100)
        
        if step_name == self.steps[-1]:
            state.status = 'completed'
        
        self.state_manager.save_state(state)
        return state
    
    def run_with_branching(self):
        """Exécuter avec branchement conditionnel"""
        # Étapes initiales
        self.execute_step('init')
        self.state_manager.create_checkpoint(self.workflow_id, 'after_init')
        
        self.execute_step('analyze')
        self.state_manager.create_checkpoint(self.workflow_id, 'after_analyze')
        
        self.execute_step('decision')
        
        # Récupérer le chemin choisi
        state = self.state_manager.load_state(self.workflow_id)
        chosen_path = state.metadata.get('chosen_path')
        
        # Exécuter le chemin approprié
        self.execute_step(chosen_path)
        self.state_manager.create_checkpoint(self.workflow_id, f'after_{chosen_path}')
        
        # Compléter
        self.execute_step('complete')
        logger.info(f"✅ Workflow avec branchement terminé")

print("✅ ConditionalWorkflow défini")

In [ ]:
# Tester le workflow conditionnel
print("\n" + "="*60)
print("WORKFLOW AVEC BRANCHEMENT CONDITIONNEL")
print("="*60)

conditional_wf = ConditionalWorkflow('workflow_002', state_manager)
conditional_wf.start({'input': 'conditional test'})
conditional_wf.run_with_branching()

# Vérifier le résultat
final_state = state_manager.load_state('workflow_002')
print(f"\n✨ Résultat:")
print(f"  Chemin choisi: {final_state.metadata.get('chosen_path')}")
print(f"  Valeur d'analyse: {final_state.data.get('analysis_result')}")
print(f"  Status: {final_state.status}")

## 7. Récapitulatif et Bonnes Pratiques

### ✅ Concepts clés

1. **State Persistence**: Sauvegarder l'état dans une base de données
2. **Checkpoints**: Points de sauvegarde pour rollback
3. **Recovery**: Reprendre après erreur
4. **History Tracking**: Audit trail de toutes les actions
5. **Conditional Branching**: Décisions basées sur l'état

### 💡 Bonnes pratiques

1. **Créer des checkpoints stratégiques**:
   - Après les étapes coûteuses
   - Avant les opérations risquées
   - Aux points de décision

2. **Gérer les erreurs gracieusement**:
   - Sauvegarder l'état en cas d'erreur
   - Logger les détails de l'erreur
   - Permettre la reprise

3. **Monitorer l'historique**:
   - Tracer toutes les actions
   - Analyser les patterns d'erreur
   - Optimiser les workflows

4. **Nettoyer les anciens états**:
   - Supprimer les workflows terminés
   - Archiver l'historique ancien
   - Gérer la taille de la DB

### 🎯 Cas d'usage en production

- **Pipelines de traitement de données**: Reprise après pannes
- **Workflows de validation**: Rollback sur échec de validation
- **Orchestration LLM**: État entre appels API multiples
- **Systèmes de paiement**: Traçabilité et rollback transactionnels
- **ETL processes**: Checkpoints entre transformations

## 📝 Exercices

### Exercice 1: Workflow de traitement de fichiers
1. Créez un workflow: upload → validate → transform → save
2. Ajoutez des checkpoints après chaque étape
3. Simulez une erreur à 'transform' et faites un rollback

### Exercice 2: Système de retry automatique
1. Implémentez une méthode `execute_with_retry(step, max_retries=3)`
2. En cas d'échec, rollback et réessayer
3. Logger chaque tentative

### Exercice 3: Workflow parallèle
1. Créez un workflow qui lance 3 branches en parallèle
2. Attendez la completion de toutes les branches
3. Mergez les résultats

### Exercice 4: Dashboard de monitoring
1. Créez une fonction pour lister tous les workflows actifs
2. Affichez leur statut et étape actuelle
3. Permettez le rollback manuel depuis le dashboard